In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 128

import skimage
import skimage.io as sio
import skimage.transform

import yuca
from yuca.ca.neural import NCA
from yuca.ca.continuous import CCA
from yuca.cppn import CPPN

from yuca.zoo.librarian import Librarian
from yuca.kernels import get_kernel

torch.set_default_dtype(torch.float32)

import IPython

from importlib import reload
reload(yuca)
reload(yuca.ca)



In [ ]:
def plot_ca(ca, my_suptitle = ""):
    
    my_cmap = lambda x: np.array(plt.get_cmap("magma")(x))
    colorify = lambda x: 1.0 - my_cmap(x)[...,:3]
    
    fig, ax = plt.subplots(1,3, figsize=(9,3), gridspec_kw={\
            "width_ratios": [0.1, 0.1, 0.7],"height_ratios": [1]})
    
    K_i = ca.id_kernel.detach().cpu().squeeze().numpy()
    K_n = ca.neighborhood_kernels.detach().cpu().squeeze().numpy()
    
    #dim = max(K_n.shape)
    
    # nearest neighbor resize
    #K_i = skimage.transform.resize(K_i,(dim,dim), order=0)
    K_i /= K_i.max()
    K_n /= K_n.max()
    
    display_kernels = (colorify(K_i), colorify(K_n))
    
    x = torch.arange(0, 1.0, 0.001).reshape(1,1,1,-1)
    
    P_x = ca.update_universe(torch.ones_like(x), x).numpy().squeeze() #ca.persistence_fns[0](x)
    G_x = ca.update_universe(torch.zeros_like(x), x).numpy().squeeze()  #ca.genesis_fns[0](x)
    xx = x.squeeze()
    
    ax[0].imshow(display_kernels[0])
    ax[1].imshow(display_kernels[1])
    
    ax[0].set_title("$K_i$")    
    ax[1].set_title("$K_n$")
    
    my_color = (1.0 - my_cmap(192)[:3], 1.0-my_cmap(128)[:3])
    
    if type(ca) == yuca.ca.neural.NCA:
        pass
    
    if (P_x - G_x).sum() == 0.0:
        ax[2].plot(xx, P_x, lw=3, label= "Growth $G$", alpha=0.85, color=my_color[0])
    else:
        ax[2].plot(xx, P_x, "--", lw=3, \
                label="Persistence $P$", alpha=0.85, color=my_color[0])
        ax[2].plot(xx, G_x, "-", lw=3, \
                label="Genesis $G$", alpha=0.85, color=my_color[1])
    
    ax[2].set_title("Update function(s)")
    
    fig.suptitle(my_suptitle, fontsize=22)
    plt.legend()
    plt.tight_layout()
    
    return fig, ax

def plot_ca_pattern(pattern, ca, my_suptitle = "", row_letter=""):
    
    my_cmap = lambda x: np.array(plt.get_cmap("magma")(x))
    colorify = lambda x: 1.0 - my_cmap(x)[...,:3]
    
    fig, ax = plt.subplots(1,4, figsize=(9,2.5), gridspec_kw={\
            "width_ratios": [0.1, 0.1, 0.6, 0.1],"height_ratios": [1]})
    
    K_i = ca.id_kernel.detach().cpu().squeeze().numpy()
    K_n = ca.neighborhood_kernels.detach().cpu().squeeze().numpy()
    
    #dim = max(K_n.shape)
    
    # nearest neighbor resize
    #K_i = skimage.transform.resize(K_i,(dim,dim), order=0)
    K_i /= K_i.max()
    K_n /= K_n.max()
    
    display_kernels = (colorify(K_i), colorify(K_n))
    
    x = torch.arange(0, 1.0, 0.001).reshape(1,1,1,-1)
    
    P_x = ca.update_universe(torch.ones_like(x), x).numpy().squeeze() #ca.persistence_fns[0](x)
    G_x = ca.update_universe(torch.zeros_like(x), x).numpy().squeeze()  #ca.genesis_fns[0](x)
    xx = x.squeeze()
    
    ax[0].imshow(display_kernels[0])
    ax[1].imshow(display_kernels[1])
    
    ax[0].set_title("$K_i$")    
    ax[1].set_title("$K_n$")
    
    my_color = (1.0 - my_cmap(192)[:3], 1.0-my_cmap(128)[:3])
    
    if type(ca) == yuca.ca.neural.NCA:
        pass
    
    if (P_x - G_x).sum() == 0.0:
        ax[2].plot(xx, P_x, lw=3, label= "Growth $G$", alpha=0.85, color=my_color[0])
    else:
        ax[2].plot(xx, P_x, "--", lw=3, \
                label="Persistence $P$", alpha=0.85, color=my_color[0])
        ax[2].plot(xx, G_x, "-", lw=3, \
                label="Genesis $G$", alpha=0.85, color=my_color[1])
    
    ax[2].set_title("Update function(s)")
    
    fig.suptitle(my_suptitle, fontsize=22)
    fig.legend(loc=[0.65, 0.450])
    
    pad_by = max(pattern.shape) // 2
    display_pattern = colorify(np.pad(pattern.squeeze(), pad_by))
    ax[3].imshow(display_pattern)
    ax[3].set_title("Glider")
    ax[0].set_ylabel(row_letter, fontsize=36, rotation=0)
    
    plt.tight_layout()
    
    return fig, ax

In [ ]:
lib = Librarian()
lib.index

In [ ]:
nca = NCA()

In [ ]:
nca_names = ["neurorbium000",\
             "neurosynorbium000",\
             "neurosingle_glider000",\
             "neurowobble_glider000"]

for name in nca_names:
    
    lib.verbose = False
    p, m = lib.load(name)

    nca.restore_config(m["ca_config"])
    print(f"\n NCA {name} :\n", f"config name: {m['ca_config']}\n", nca.weights_layer)
    
    fig, ax = plot_ca_pattern(p, nca,  my_suptitle=f"{name}", row_letter="  ") 

    plt.show()

In [ ]:
m